<a href="https://colab.research.google.com/github/hongsamhc2/AutoTradingSystem/blob/master/%EC%A1%B0%ED%9B%88%EC%B0%BD/regression_models_train_XGBRegressor_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [9]:
df = pd.read_csv('/content/drive/My Drive/datasets/stock/ta_DA000020.csv')

In [10]:
df['date'] = df['date'].astype('str')
df.set_index(df['date'],inplace=True)
df['label'] = df['close'].shift(-1)

In [11]:
cols = df.columns
for col in cols:
  if df[col].sum() == 0 :
    df.drop(col,axis=1,inplace=True)

In [54]:
df_ = df.dropna().copy()
df_.isnull().sum()

Unnamed: 0    0
date          0
open          0
high          0
low           0
             ..
WMA_20        0
WMA_30        0
WMA_60        0
WMA_120       0
label         0
Length: 110, dtype: int64

In [55]:
# make y_label and X_dataset

y_label = df_['label'].copy()
X_dataset = df_.drop(['Unnamed: 0','label'],axis=1).copy()
X_dataset['date'] = X_dataset['date'].astype(int)

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score,accuracy_score

In [88]:
def get_model(X,y,model_param):
  for i in model_param:
    if i == 'minmax':
      scaled = MinMaxScaler()
      X_scaled = scaled.fit_transform(X)
      X_train1, X_test1, y_train1,y_test1 = train_test_split(X_scaled,y,train_size = 0.9,shuffle=False)
      scores1 = model_score(X_train1,X_test1,y_train1,y_test1)
      print('scaler : ' ,i)
      for score in scores1:
        print(score[0] + ' score : ', np.round(score[1],3))
    elif i == 'standard':
      scaled = StandardScaler()
      X_scaled = scaled.fit_transform(X)
      X_train1, X_test1, y_train1,y_test1 = train_test_split(X_scaled,y,train_size = 0.9,shuffle=False)
      scores1 = model_score(X_train1,X_test1,y_train1,y_test1)
      print('scaler : ' ,i)
      for score in scores1:
        print(score[0] + ' score : ', np.round(score[1],3))

    



In [89]:
def model_score(X_train_,X_test_,y_train_,y_test_):
  lin = LinearRegression()
  ridge = Ridge()
  lasso = Lasso()
  random = RandomForestRegressor()
  xgb = XGBRegressor()
  lgbm = LGBMRegressor()
  svr = SVR()

  models = [lin,ridge,lasso,random,xgb,lgbm,svr]
  result = []
  for model in models:
    model.fit(X_train_,y_train_)
    pred = model.predict(X_test_)
    score = r2_score(y_test_,pred)
    list = [model.__class__.__name__ , score]
    result.append(list)
  return result
    

  


  

In [90]:
get_model(X_dataset,y_label,['minmax','standard'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 307832522.15005684, tolerance: 143002786.20836475
  positive)


[08:24:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
scaler :  minmax
LinearRegression score :  0.984
Ridge score :  0.984
Lasso score :  0.987
RandomForestRegressor score :  0.935
XGBRegressor score :  0.978
LGBMRegressor score :  0.604
SVR score :  0.162


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2296256073.1985726, tolerance: 143002786.20836475
  positive)


[08:26:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
scaler :  standard
LinearRegression score :  0.984
Ridge score :  0.983
Lasso score :  0.898
RandomForestRegressor score :  0.926
XGBRegressor score :  0.978
LGBMRegressor score :  0.955
SVR score :  0.242


In [95]:
xgb = XGBRegressor()

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}
xgb_grid = GridSearchCV(xgb,
                        parameters,
                        cv = 10,
                        n_jobs = -1,
                        verbose=True)

In [97]:
X_train,X_test,y_train,y_test = train_test_split(X_dataset,y_label,train_size = 0.9, shuffle = False)
xgb_grid.fit(X_train,y_train)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 19.5min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 39.6min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg...
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'colsample_bytree': [0.7],
                         'learning_rate': [0.03, 0.05, 0.07],
                         '

In [98]:
xgb_grid.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.03,
 'max_depth': 6,
 'min_child_weight': 4,
 'n_estimators': 500,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.7}

In [99]:
xgb = XGBRegressor(colsample_bytree=0.7,learning_rate=0.03,max_depth=6,min_child_weight=4,n_estimators=500,nthread=4,objective='reg:linear',silent=1,subsample=0.7)

In [100]:
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.03, max_delta_step=0,
             max_depth=6, min_child_weight=4, missing=None, n_estimators=500,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [103]:
pred = xgb.predict(X_test)

print(r2_score(y_test,pred))


0.9793739395504665
